In [13]:
import cv2
import numpy as np

def process_image1(image):
    # Load YOLO
    net = cv2.dnn.readNet("./darknet/yolov3.weights", "./darknet/cfg/yolov3.cfg")

    # Load class labels
    classes = []
    with open("./darknet/data/coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Get the image dimensions
    height, width, _ = image.shape

    # Preprocess the image
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob
    net.setInput(blob)

    # Forward pass through the network
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)

    # Extract bounding box coordinates and class predictions
    boxes = []
    confidences = []
    class_ids = []
    labels = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
                labels.append(classes[class_id])

    # Apply non-maximum suppression to eliminate overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels
    for i in indices:
        # i = i[0]
        x, y, w, h = boxes[i]
        label = labels[i]
        confidence = confidences[i]
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(image, f"{label}: {confidence:.2f}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image, labels

# Test the function
# input_image = cv2.imread("./image.jpg")
# output_image, detected_labels = process_image1(input_image)

# # Display the output image
# cv2.imshow("Object Detection", output_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# # Print the detected labels
# print("Detected Labels:")
# for label in detected_labels:
#     print(label)


In [14]:
from flask import Flask, request, send_file, jsonify
import base64
from PIL import Image
import io
import numpy as np
import cv2
import tensorflow as tf
# from tensorflow.keras.applications import EfficientDetModel
app = Flask(__name__)

# model = tf.keras.applications.MobileNetV2(weights='imagenet')
# model = tf.keras.applications.EfficientNetB0(weights='imagenet')
def detect_edges(img):
    edges = cv2.Canny(img, 100, 200)
    return edges


@app.route('/process-image1', methods=['POST'])
def process_image_route():
    image_file = request.files.get('image')
    image = cv2.imdecode(np.frombuffer(image_file.read(), np.uint8), cv2.IMREAD_COLOR)
    image1 = image.copy()

    # Process the image
    processed_image, detected_labels = process_image1(image1)

    # Convert the processed image to JPEG format
    ret, jpeg = cv2.imencode('.jpg', processed_image)
    encoded_image = jpeg.tobytes()

    # Create a response JSON object
    response = {
        'labels': detected_labels,
        'image': base64.b64encode(encoded_image).decode('utf-8')
    }

    # Return the response as JSON
    return jsonify(response)

    # Return the encoded image as a response
    # return Response(encoded_image, mimetype='image/jpeg')
if __name__ == '__main__':
    #run app in debug mode on port 5000
    app.run(host='0.0.0.0', port=5000)
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.1.7:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/May/2023 18:35:32] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:36:09] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:37:19] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:38:21] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:40:28] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:45:00] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:47:07] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:49:09] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:50:48] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:51:07] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:51:34] "POST /process-image1 HTTP/1.1" 200 -
192.168.1.5 - - [14/May/2023 18:55:46] "POST /process-image1 HTT

In [8]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("./darknet/yolov3.weights", "./darknet/cfg/yolov3.cfg")

# Load class labels
classes = []
with open("./darknet/data/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Set the input image and blob dimensions
image = cv2.imread("./test.jpg")
height, width, _ = image.shape

# Preprocess the image
blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)

# Set the input blob
net.setInput(blob)

# Forward pass through the network
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
outputs = net.forward(output_layers)

# output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# outputs = net.forward(output_layers)

# Extract bounding box coordinates and class predictions
boxes = []
confidences = []
class_ids = []
for output in outputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)

# Apply non-maximum suppression to eliminate overlapping bounding boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
# indices = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), 0.5, 0.4)
# Draw bounding boxes and labels
for i in indices:
    # i = i[0]
    x, y, w, h = boxes[i]
    label = classes[class_ids[i]]
    confidence = confidences[i]
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)
    cv2.putText(image, f"{label}: {confidence:.2f}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the output image
cv2.imshow("Object Detection", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[13 12 17  9 19  3 22 15 28  1 29 24  0 25]
